In [ ]:
import tcxparser
import pandas as pd
from datetime import datetime
from os import listdir
from os.path import join

def get_hr_data(tcx_path):
    tcx = tcxparser.TCXParser(tcx_path)
    hr_values = tcx.hr_values()
    time_values = [datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.000Z') for t in tcx.time_values()]
    elapsed = [(tv - time_values[0]).seconds for tv in time_values]
    return pd.DataFrame(data={'elapsed': elapsed, 'hr': hr_values})

def load_data(dir_path):
    tcx_filenames = [f for f in listdir(dir_path) if f.endswith('.tcx')]
    return [get_hr_data(join(dir_path, f)) for f in tcx_filenames]

In [ ]:
load_data('data')